## Teste Race log
### Santiago Davila


Vou tratar o log como um texto a ser carregado como um dataframe Pandas, e assim usar os recursos de analise de dados que achei úteis nesse problema. 

In [176]:
#Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import datetime
import time


#### Carga do arquivo de texto
Utilizo nomes descritivos para as colunas, elimino aquela adicional sem informação.

In [177]:
#Nomes de colunas a serem utilizadas
column_names = ["HORA", "CODIGO", "TRACO", "NOME", "NUM_VOLTA", "TEMPO_VOLTA", "VEL_MEDIA_VOLTA" ]
#Carrego o log desconsiderando a primeira linha
log_df = pd.read_csv('race_log.txt', delimiter=r"\s+", skiprows=1, names=column_names)
#Apago coluna
del log_df["TRACO"]
#Visualizo primeiros resultados
log_df.head(8)

,HORA,CODIGO,NOME,NUM_VOLTA,TEMPO_VOLTA,VEL_MEDIA_VOLTA
0,23:49:08.277,38,F.MASSA,1,1:02.852,"44,275"
1,23:49:10.858,33,R.BARRICHELLO,1,1:04.352,"43,243"
2,23:49:11.075,2,K.RAIKKONEN,1,1:04.108,"43,408"
3,23:49:12.667,23,M.WEBBER,1,1:04.414,"43,202"
4,23:49:30.976,15,F.ALONSO,1,1:18.456,"35,47"
5,23:50:11.447,38,F.MASSA,2,1:03.170,"44,053"
6,23:50:14.860,33,R.BARRICHELLO,2,1:04.002,"43,48"
7,23:50:15.057,2,K.RAIKKONEN,2,1:03.982,"43,493"


#### Análise Exploratório
Confiro a quantidade de pilotos, o formato dos dados e se tem valores sem formato correto

In [178]:
log_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 6 columns):
HORA               23 non-null object
CODIGO             23 non-null int64
NOME               23 non-null object
NUM_VOLTA          23 non-null int64
TEMPO_VOLTA        23 non-null object
VEL_MEDIA_VOLTA    23 non-null object
dtypes: int64(2), object(4)
memory usage: 1.2+ KB


Identificamos que não temos valores nulos, e que a hora não está sendo tratada como DATA, irei converter essa informação mas a falta do dia, mes e ano fazem que essa informação não possa ser usada desse jeito. Caso seja necessário teriamos que reconstruir a coluna como timestamps a partir da hora da primeira volta, já que uma corrida pode iniciar num dia e terminar no outro

In [179]:
#Conversão a data
log_df['HORA'] = pd.to_datetime(log_df['HORA'])

In [180]:
#Função para aplicar a conversão de tempo
def convert_to_ms(time_string):
    t = time.strptime(time_string, '%M:%S.%f')
    seconds = datetime.timedelta(hours=t.tm_hour,minutes=t.tm_min,seconds=t.tm_sec).total_seconds()
    ms = int(time_string.split('.')[1])
    return (1000 * seconds) + ms

#Convertemos o texto de tempo de volta em tempo 
log_df['TEMPO'] = log_df['TEMPO_VOLTA'].apply(lambda x: convert_to_ms(x))


In [181]:
log_df.head()

,HORA,CODIGO,NOME,NUM_VOLTA,TEMPO_VOLTA,VEL_MEDIA_VOLTA,TEMPO
0,2019-05-05 23:49:08.277,38,F.MASSA,1,1:02.852,"44,275",62852.0
1,2019-05-05 23:49:10.858,33,R.BARRICHELLO,1,1:04.352,"43,243",64352.0
2,2019-05-05 23:49:11.075,2,K.RAIKKONEN,1,1:04.108,"43,408",64108.0
3,2019-05-05 23:49:12.667,23,M.WEBBER,1,1:04.414,"43,202",64414.0
4,2019-05-05 23:49:30.976,15,F.ALONSO,1,1:18.456,"35,47",78456.0


In [182]:
log_df['NOME'].unique()

array(['F.MASSA', 'R.BARRICHELLO', 'K.RAIKKONEN', 'M.WEBBER', 'F.ALONSO',
       'S.VETTEL', 'F.MASS'], dtype=object)

In [183]:
#Corrigimos o dado errado para manter a consistencia
log_df['NOME'] = log_df['NOME'].replace('F.MASS', 'F.MASSA')
log_df['NOME'].unique()

array(['F.MASSA', 'R.BARRICHELLO', 'K.RAIKKONEN', 'M.WEBBER', 'F.ALONSO',
       'S.VETTEL'], dtype=object)

#### Tratamento dos dados a serem consultados
- Posição Chegada
- Código Piloto 
- Nome Piloto 
- Qtde Voltas Completadas 
- Tempo Total de Prova



In [184]:
def convert_to_timestamp(time_ms):
    str_time = time.strftime('%H:%M:%S', time.gmtime(time_ms))
    ms = time_ms % 1000
    rest_sec = (time_ms-ms)/1000 
    sec = rest_sec % 60
    rest_min = (rest_sec - sec)/60
    minute = rest_min % 60
    rest_hour = (rest_min-minute)/60
    hour = rest_hour % 60
    return time.strftime('%H:%M:%S', time.gmtime(rest_sec)) + '.' + str(ms/1000).split('.')[1]


In [185]:
group_df = log_df \
.groupby('CODIGO') \
.agg(
    {
        #Voltas completadas
        'NUM_VOLTA': 'max',
        #Tempo total de prova
        'TEMPO': "sum",
        # min, first, and number of unique dates per group
        'NOME': 'first',
    }
)
group_df = group_df.reset_index()
group_df = group_df.sort_values(by='NUM_VOLTA', ascending=False).sort_values(by='TEMPO', ascending=True)
group_df = group_df.reset_index()

del group_df['index']
group_df['POSICAO']= group_df.index +1

group_df['TEMPO_TOTAL'] = group_df['TEMPO'].apply(lambda x: convert_to_timestamp(x))

group_df.head(10)

,CODIGO,NUM_VOLTA,TEMPO,NOME,POSICAO,TEMPO_TOTAL
0,38,4,251578.0,F.MASSA,1,00:04:11.578
1,2,4,255153.0,K.RAIKKONEN,2,00:04:15.153
2,33,4,256080.0,R.BARRICHELLO,3,00:04:16.08
3,23,4,257722.0,M.WEBBER,4,00:04:17.722
4,15,4,294221.0,F.ALONSO,5,00:04:54.221
5,11,3,387276.0,S.VETTEL,6,00:06:27.276
